# Video Frame Access

Original Author: Tim Crone  
Adapted from https://github.com/tjcrone/rte-camhd/blob/master/tim/image_sliders.ipynb  
    
Example of using widgets to interact with frame grabs from the OOI HD video camera at ASHES Hydrothermal Vent. There are over 100 TB of video data captured on the raw archive to date. This notebook provides access to frames within each video file, directly over the web and without the need to download a local copy. This technique opens the door to [rapid image analysis](https://github.com/tjcrone/rte-camhd/blob/master/tim/floc.ipynb) and scientific discovery. 

#### Import your libraries and set up the environment

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pycamhd as camhd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Get a list of remote CamHD files to process

In [2]:
db_url = 'https://raw.githubusercontent.com/tjcrone/rte-camhd/master/tim/dbcamhd.json'
dbcamhd = pd.read_json(db_url, orient="records", lines=True)
dbcamhd.tail()

,deployment,filename,filesize,frame_count,moov,timestamp
8423,4,https://rawdata.oceanobservatories.org/files/R...,596503857,26981,True,1522627201
8424,4,https://rawdata.oceanobservatories.org/files/R...,595973297,26957,True,1522638002
8425,4,https://rawdata.oceanobservatories.org/files/R...,596481805,26980,True,1522648801
8426,4,https://rawdata.oceanobservatories.org/files/R...,596459753,26979,True,1522659601
8427,4,https://rawdata.oceanobservatories.org/files/R...,595840657,26951,True,1522670402


In [3]:
print("Total files: %i" % len(dbcamhd))
print("Total frames: %i" % dbcamhd.frame_count.sum())

Total files: 8428
Total frames: 227632424


In [4]:
filenames = list(dbcamhd.filename[(dbcamhd.deployment == 2) & (dbcamhd.frame_count > 5000) & (dbcamhd.frame_count < 30000)])
filenames.sort()
filenames[0]

'https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2015/11/19/CAMHDA301-20151119T210000Z.mov'

#### Set up plotting function

In [5]:
def show_image(file_number, frame_number):
    filename = filenames[file_number]
    plt.rc('figure', figsize=(10, 5))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(filename, frame_number, 'rgb24')
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    plt.title('%s %s' % (filename[84:], frame_number));

#### Interact using ipywidgets

In [6]:
from ipywidgets import interact
from ipywidgets import IntSlider
file_slider = IntSlider(min=0, max=len(filenames)-1, step=1, value=0, continuous_update=False)
frame_slider = IntSlider(min=0, max=camhd.get_frame_count(filenames[0]), step=1, value=0, continuous_update=False)

def update_range(*args):
    frame_slider.max = camhd.get_frame_count(filenames[file_slider.value])-1
    if frame_slider.value > frame_slider.max:
        frame_slider.value = frame_slider.max
file_slider.observe(update_range, 'value')

interact(show_image, file_number=file_slider, frame_number=frame_slider);

interactive(children=(IntSlider(value=0, continuous_update=False, description='file_number', max=1884), IntSli…

### References  
- [PyCamHD](https://github.com/tjcrone/pycamhd)  
- [CamHD Raw Data Archive](https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301)  
- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)  